In [372]:
import pandas as pd
import math

In [228]:
week1 = pd.read_csv('/Users/alexmcgraw/Documents/Adv_Predictive_Modeling/Final_Presentation/nfl-big-data-bowl-2021/week1.csv')

In [229]:

# Adding a player category, instead of just position
cat_item = {'Offense': ['QB', 'RB', 'FB', 'WR', 'TE', 'HB'], 
            'Defense': ['OLB', 'MLB', 'LB', 'ILB', 'CB', 'DE', 'DT', 'NT', 'DB', 'S', 'SS', 'FS'], 
            'Special': ['K', 'P', 'LS']}
item_cat = {w: k for k, v in cat_item.items() for w in v}

week1['side'] = week1['position'].map(item_cat)

In [230]:
mask = (week1['position'] == 'CB') | (week1['position'] == 'WR')
df = week1[mask]


In [105]:
df['event'].value_counts()

None                         340750
ball_snap                      5728
pass_forward                   5333
pass_arrived                   4304
pass_outcome_caught            3075
first_contact                  2576
tackle                         2550
pass_outcome_incomplete        1997
play_action                     904
out_of_bounds                   658
line_set                        578
qb_sack                         286
man_in_motion                   195
pass_outcome_interception       186
touchdown                       139
pass_tipped                     109
fumble                           94
pass_outcome_touchdown           83
fumble_defense_recovered         72
qb_strip_sack                    69
fumble_offense_recovered         56
shift                            49
handoff                          36
penalty_flag                     25
pass_shovel                      24
run                              14
qb_spike                         11
touchback                   

In [239]:
play75 = df[(df['playId'] == 75) & ((df['event'] == 'pass_forward') | (df['event'] == 'pass_arrived'))]
pass_ForArr = df[((df['event'] == 'pass_forward') | (df['event'] == 'pass_arrived'))]# & ((df['nflId'] == 2495454.0) |\
                                                                                     #   (df['nflId'] == 2552689.0))]

"""onlySnap = play75[play75['event'] == 'ball_snap']
onlySnap = onlySnap[['time','x','y','event','nflId','frameId','position']]
onlySnap.iloc[:10]"""

pass_ForArr['position'].value_counts()

CB    4926
WR    4711
Name: position, dtype: int64

In [233]:
for i in range(len(play75)):
    if play75['event'].iloc[i] == 'ball_snap':
        
        #trying to create a mini loop over only ball snap, to then calculate starting euclidean distance between WRs and CBs
        
        onlySnap = play75[play75['event'] == 'ball_snap']
        onlySnap = onlySnap[['x','y','event','nflId','frameId','position']]
        length = len(onlySnap)
        
        for idx, val in enumerate(play75.iloc[:length]):
            
            #Trying to use a list comprehension to get the x,y coordinates of the WRs and CBs in lists, to then calc.
            #their euclidean distances between one another.
            

SyntaxError: unexpected EOF while parsing (<ipython-input-233-c6d083740f5d>, line 14)

In [90]:
#IDEA. Create new mini DFs that have just the info I need to calculate each, and then append the output values after
#all the calculations to another DF. for tracking and all that

In [128]:
mask_WR = onlySnap['position'] == 'WR'
WR = onlySnap[mask_WR]
WR

mask_CB = onlySnap['position'] == 'CB'
CB = onlySnap[mask_CB]
CB

,time,x,y,event,nflId,frameId,position
148,2018-09-07T01:07:15.599Z,82.63,20.50,ball_snap,2552689.0,11,CB
149,2018-09-07T01:07:15.599Z,83.92,43.44,ball_snap,2555383.0,11,CB


In [146]:
play75

,time,x,y,s,a,dis,o,dir,event,nflId,displayName,jerseyNumber,position,frameId,team,gameId,playId,playDirection,route,side
492,2018-09-07T01:07:18.099Z,80.69,44.91,3.11,6.67,0.34,155.11,297.88,pass_forward,2495454.0,Julio Jones,11.0,WR,36,away,2018090600,75,left,HITCH,Offense
494,2018-09-07T01:07:18.099Z,82.65,34.56,6.35,1.21,0.64,262.06,272.78,pass_forward,2533040.0,Mohamed Sanu,12.0,WR,36,away,2018090600,75,left,HITCH,Offense
498,2018-09-07T01:07:18.099Z,79.51,20.00,2.99,0.69,0.30,78.25,259.75,pass_forward,2552689.0,Ronald Darby,21.0,CB,36,home,2018090600,75,left,NaN,Defense
499,2018-09-07T01:07:18.099Z,76.53,44.93,4.68,2.18,0.47,182.14,277.48,pass_forward,2555383.0,Jalen Mills,31.0,CB,36,home,2018090600,75,left,NaN,Defense
646,2018-09-07T01:07:19.200Z,81.11,47.87,4.23,1.63,0.42,53.31,22.58,pass_arrived,2495454.0,Julio Jones,11.0,WR,47,away,2018090600,75,left,HITCH,Offense
648,2018-09-07T01:07:19.200Z,77.66,35.52,2.99,3.53,0.31,78.88,305.48,pass_arrived,2533040.0,Mohamed Sanu,12.0,WR,47,away,2018090600,75,left,HITCH,Offense
652,2018-09-07T01:07:19.200Z,77.11,17.30,4.83,3.10,0.47,167.90,193.72,pass_arrived,2552689.0,Ronald Darby,21.0,CB,47,home,2018090600,75,left,NaN,Defense
653,2018-09-07T01:07:19.200Z,74.40,47.11,4.05,4.64,0.38,69.31,13.45,pass_arrived,2555383.0,Jalen Mills,31.0,CB,47,home,2018090600,75,left,NaN,Defense


In [243]:
#USE THIS DAMN THING SO RIGHT


pivot = pd.pivot_table(pass_ForArr, values = ['x', 'y'], index=['gameId', 'playId','event'], columns = ['position', 'nflId'])
pivot

x                                \
position                              CB                                 
nflId                          306.0     1302.0    4365.0    80668.0     
gameId     playId event                                                  
2018090600 75     pass_arrived       NaN       NaN       NaN       NaN   
                  pass_forward       NaN       NaN       NaN       NaN   
           146    pass_arrived       NaN       NaN       NaN       NaN   
                  pass_forward       NaN       NaN       NaN       NaN   
           168    pass_forward       NaN       NaN       NaN       NaN   
           190    pass_arrived       NaN       NaN       NaN       NaN   
                  pass_forward       NaN       NaN       NaN       NaN   
           320    pass_arrived       NaN       NaN       NaN       NaN   
                  pass_forward       NaN       NaN       NaN       NaN   
           344    pass_forward       NaN       NaN       NaN       NaN   
           402    pass_arrived       NaN       NaN       NaN       NaN   
                  pass_forward       NaN       NaN       NaN       NaN   
           492    pass_arrived       NaN       NaN       NaN       NaN   
                  pass_forward       NaN       NaN       NaN       NaN   
           521    pass_arrived       NaN       NaN       NaN       NaN   
                  pass_forward       NaN       NaN       NaN       NaN   
           545    pass_forward       NaN       NaN       NaN       NaN   
           636    pass_forward       NaN       NaN       NaN       NaN   
           752    pass_arrived       NaN       NaN       NaN       NaN   
                  pass_forward       NaN       NaN       NaN       NaN   
           889    pass_forward       NaN       NaN       NaN       NaN   
           949    pass_arrived       NaN       NaN       NaN       NaN   
                  pass_forward       NaN       NaN       NaN       NaN   
           992    pass_arrived       NaN       NaN       NaN       NaN   
                  pass_forward       NaN       NaN       NaN       NaN   
           1037   pass_arrived       NaN       NaN       NaN       NaN   
                  pass_forward       NaN       NaN       NaN       NaN   
           1061   pass_arrived       NaN       NaN       NaN       NaN   
                  pass_forward       NaN       NaN       NaN       NaN   
           1085   pass_arrived       NaN       NaN       NaN       NaN   
...                                  ...       ...       ...       ...   
2018091001 3441   pass_forward       NaN     77.90       NaN       NaN   
           3463   pass_arrived       NaN     74.81       NaN       NaN   
                  pass_forward       NaN     75.02       NaN       NaN   
           3487   pass_arrived       NaN     74.73       NaN       NaN   
                  pass_forward       NaN     76.01       NaN       NaN   
           3511   pass_arrived       NaN     30.44       NaN       NaN   
                  pass_forward       NaN     34.37       NaN       NaN   
           3580   pass_forward       NaN       NaN       NaN       NaN   
           3602   pass_arrived       NaN       NaN       NaN       NaN   
                  pass_forward       NaN       NaN       NaN       NaN   
           3626   pass_forward       NaN       NaN       NaN       NaN   
           3715   pass_arrived       NaN       NaN       NaN       NaN   
                  pass_forward       NaN       NaN       NaN       NaN   
           3765   pass_forward       NaN       NaN       NaN       NaN   
           3787   pass_arrived       NaN       NaN       NaN       NaN   
                  pass_forward       NaN       NaN       NaN       NaN   
           3904   pass_arrived       NaN     64.49       NaN       NaN   
                  pass_forward       NaN     65.44       NaN       NaN   
           3928   pass_arrived       NaN     59.60       NaN       NaN   
                  pass_forward       NaN   

In [295]:
print(len(pivot))

1587


In [297]:
#CB_x = pivot.xs(('x', 'CB', 2552689.0), axis=1)
df1 = pd.DataFrame()
print(len(pivot))
for play, event in pivot.groupby(level=1):
    #print(len(event))
    
    if len(event) < 2:
        #print statements are used to check if working. This loop deletes rows that don't have a pass arrived
        
        print((pivot.index.get_level_values(1) == 60).sum())
        
        pivot.drop(play,level=1, inplace=True)
        
        print((pivot.index.get_level_values(1) == 60).sum())
        

1587


8

In [380]:
#importing targeted receivers and cb_wr dataset from dylan

targetedReceiver = pd.read_csv('/Users/alexmcgraw/Documents/Adv_Predictive_Modeling/APM-Term-Project/targetedReceiver.csv')
cb_wr = pd.read_csv('/Users/alexmcgraw/Documents/Adv_Predictive_Modeling/APM-Term-Project/cb_wr.csv')

mask = ((cb_wr['position'] == 'CB') | (cb_wr['position'] == 'WR') & (cb_wr['event'] == 'pass_forward') | (cb_wr['event']\
                                                                                                        == 'pass_arrived'))
#getting only the pass forward and arrived events for CB and WR
cb_wr_passes = cb_wr[mask]

#only one game
game1 = cb_wr_passes[(cb_wr_passes['gameId'] == 2018090600)]
game1 = game1[['time','playId', 'x', 'y', 'event','nflId','position','team']]

game1Receivers = targetedReceiver[(targetedReceiver['gameId'] == 2018090600)]

In [385]:
game1.head()

,time,playId,x,y,event,nflId,position,team
0,2018-09-07T01:07:18.099Z,75,80.69,44.91,pass_forward,2495454.0,WR,away
1,2018-09-07T01:07:18.099Z,75,82.65,34.56,pass_forward,2533040.0,WR,away
2,2018-09-07T01:07:18.099Z,75,79.51,20.00,pass_forward,2552689.0,CB,home
3,2018-09-07T01:07:18.099Z,75,76.53,44.93,pass_forward,2555383.0,CB,home
4,2018-09-07T01:07:19.200Z,75,81.11,47.87,pass_arrived,2495454.0,WR,away


In [386]:
game1Receivers.head()

,gameId,playId,targetNflId
0,2018090600,75,2495454.0
1,2018090600,146,2560854.0
2,2018090600,168,2543583.0
3,2018090600,190,2495454.0
4,2018090600,256,2543583.0


In [460]:
#Merging game and targeted receivers
game_receiver_merge = game1.merge(game1Receivers, how='outer', left_on = ['playId', 'nflId'], right_on = ['playId',
                                                                                                          'targetNflId'])
#replacing NaN with None
game_receiver_merge = game_receiver_merge.where(pd.notnull(game_receiver_merge), None)

#getting rid of instances where there is a WR that isn't the targeted one
game_receiver_merge = game_receiver_merge[(((game_receiver_merge['position'] == 'WR') & (game_receiver_merge['nflId'] == game_receiver_merge
                                                                 ['targetNflId'])) | \
                     (game_receiver_merge['position'] == 'CB'))]

#zipping x and y
game_receiver_merge['x,y'] = list(zip(game_receiver_merge['x'], game_receiver_merge['y']))

#dropping x and y columns
game_receiver_merge = game_receiver_merge.drop(['x', 'y'], axis=1)

#gets rid of scientific notation
pd.set_option('display.float_format', lambda x: '%.0f' % x) 

game_receiver_merge.head()

,time,playId,event,nflId,position,team,gameId,targetNflId,"x,y"
0,2018-09-07T01:07:18.099Z,75,pass_forward,2495454,WR,away,2018090600,2495454,"(80.69, 44.91)"
1,2018-09-07T01:07:19.200Z,75,pass_arrived,2495454,WR,away,2018090600,2495454,"(81.11, 47.87)"
4,2018-09-07T01:07:18.099Z,75,pass_forward,2552689,CB,home,None,None,"(79.51, 20.0)"
5,2018-09-07T01:07:19.200Z,75,pass_arrived,2552689,CB,home,None,None,"(77.11, 17.3)"
6,2018-09-07T01:07:18.099Z,75,pass_forward,2555383,CB,home,None,None,"(76.53, 44.93)"


In [461]:
# FIX SO WHEN ITERATING THRU EVERY GAME

#unique play ids
plays = pd.Series(game_receiver_merge['playId'].unique())

#creating a new DF, that only has plays that have both a WR and CB, bc sometimes plays have CB but no WR, bc none targeted
merge_fixed = pd.DataFrame(columns=game_receiver_merge.columns)

for play in plays:
    df_temp = game_receiver_merge[(game_receiver_merge['playId'] == play)]
    unique_pos = df_temp['position'].unique()
    if len(unique_pos) == 2:
        merge_fixed = merge_fixed.append(df_temp)
        
merge_fixed        


,time,playId,event,nflId,position,team,gameId,targetNflId,"x,y"
0,2018-09-07T01:07:18.099Z,75,pass_forward,2495454,WR,away,2018090600,2495454,"(80.69, 44.91)"
1,2018-09-07T01:07:19.200Z,75,pass_arrived,2495454,WR,away,2018090600,2495454,"(81.11, 47.87)"
4,2018-09-07T01:07:18.099Z,75,pass_forward,2552689,CB,home,None,None,"(79.51, 20.0)"
5,2018-09-07T01:07:19.200Z,75,pass_arrived,2552689,CB,home,None,None,"(77.11, 17.3)"
6,2018-09-07T01:07:18.099Z,75,pass_forward,2555383,CB,home,None,None,"(76.53, 44.93)"
7,2018-09-07T01:07:19.200Z,75,pass_arrived,2555383,CB,home,None,None,"(74.4, 47.11)"
10,2018-09-07T01:09:12.700Z,146,pass_forward,2552689,CB,home,None,None,"(38.1, 10.36)"
11,2018-09-07T01:09:13.200Z,146,pass_arrived,2552689,CB,home,None,None,"(37.4, 9.64)"
12,2018-09-07T01:09:12.700Z,146,pass_forward,2555383,CB,home,None,None,"(41.28, 38.93)"
13,2018-09-07T01:09:13.200Z,146,pass_arrived,2555383,CB,home,None,None,"(43.3, 37.96)"


In [502]:
#merging on itself
merged2_fixed = merge_fixed.merge(merge_fixed, on=['time', 'event'])
merged2_fixed.head()


,time,playId_x,event,nflId_x,position_x,team_x,gameId_x,targetNflId_x,"x,y_x",playId_y,nflId_y,position_y,team_y,gameId_y,targetNflId_y,"x,y_y"
0,2018-09-07T01:07:18.099Z,75,pass_forward,2495454,WR,away,2018090600,2495454,"(80.69, 44.91)",75,2495454,WR,away,2018090600,2495454,"(80.69, 44.91)"
1,2018-09-07T01:07:18.099Z,75,pass_forward,2495454,WR,away,2018090600,2495454,"(80.69, 44.91)",75,2552689,CB,home,None,None,"(79.51, 20.0)"
2,2018-09-07T01:07:18.099Z,75,pass_forward,2495454,WR,away,2018090600,2495454,"(80.69, 44.91)",75,2555383,CB,home,None,None,"(76.53, 44.93)"
3,2018-09-07T01:07:18.099Z,75,pass_forward,2552689,CB,home,None,None,"(79.51, 20.0)",75,2495454,WR,away,2018090600,2495454,"(80.69, 44.91)"
4,2018-09-07T01:07:18.099Z,75,pass_forward,2552689,CB,home,None,None,"(79.51, 20.0)",75,2552689,CB,home,None,None,"(79.51, 20.0)"


In [503]:
#keeping rows where the player positions aren't the same (so no pairings between two WR or CB)
merged2_fixed = merged2_fixed[((merged2_fixed['position_x'] != merged2_fixed['position_y']))]
      
    
merged2_fixed.iloc[:10]


,time,playId_x,event,nflId_x,position_x,team_x,gameId_x,targetNflId_x,"x,y_x",playId_y,nflId_y,position_y,team_y,gameId_y,targetNflId_y,"x,y_y"
1,2018-09-07T01:07:18.099Z,75,pass_forward,2495454,WR,away,2018090600,2495454,"(80.69, 44.91)",75,2552689,CB,home,None,None,"(79.51, 20.0)"
2,2018-09-07T01:07:18.099Z,75,pass_forward,2495454,WR,away,2018090600,2495454,"(80.69, 44.91)",75,2555383,CB,home,None,None,"(76.53, 44.93)"
3,2018-09-07T01:07:18.099Z,75,pass_forward,2552689,CB,home,None,None,"(79.51, 20.0)",75,2495454,WR,away,2018090600,2495454,"(80.69, 44.91)"
6,2018-09-07T01:07:18.099Z,75,pass_forward,2555383,CB,home,None,None,"(76.53, 44.93)",75,2495454,WR,away,2018090600,2495454,"(80.69, 44.91)"
10,2018-09-07T01:07:19.200Z,75,pass_arrived,2495454,WR,away,2018090600,2495454,"(81.11, 47.87)",75,2552689,CB,home,None,None,"(77.11, 17.3)"
11,2018-09-07T01:07:19.200Z,75,pass_arrived,2495454,WR,away,2018090600,2495454,"(81.11, 47.87)",75,2555383,CB,home,None,None,"(74.4, 47.11)"
12,2018-09-07T01:07:19.200Z,75,pass_arrived,2552689,CB,home,None,None,"(77.11, 17.3)",75,2495454,WR,away,2018090600,2495454,"(81.11, 47.87)"
15,2018-09-07T01:07:19.200Z,75,pass_arrived,2555383,CB,home,None,None,"(74.4, 47.11)",75,2495454,WR,away,2018090600,2495454,"(81.11, 47.87)"
21,2018-09-07T01:09:12.700Z,146,pass_forward,2552689,CB,home,None,None,"(38.1, 10.36)",146,2560854,WR,away,2018090600,2560854,"(45.51, 36.91)"
25,2018-09-07T01:09:12.700Z,146,pass_forward,2555383,CB,home,None,None,"(41.28, 38.93)",146,2560854,WR,away,2018090600,2560854,"(45.51, 36.91)"


In [582]:
#Calculating the euclidean distance between the players
    
def euc_dist(item):
    return math.sqrt(((item['x,y_x'][0] - item['x,y_y'][0])**2) + 
                                     ((item['x,y_x'][1] - item['x,y_y'][1])**2))

merged2_fixed['Euc_dist'] = merged2_fixed.apply(euc_dist, axis=1)
    
merged2_fixed.head()


,time,playId_x,event,nflId_x,position_x,team_x,gameId_x,targetNflId_x,"x,y_x",playId_y,nflId_y,position_y,team_y,gameId_y,targetNflId_y,"x,y_y",Euc_dist
1,2018-09-07T01:07:18.099Z,75,pass_forward,2495454,WR,away,2018090600,2495454,"(80.69, 44.91)",75,2552689,CB,home,None,None,"(79.51, 20.0)",25
2,2018-09-07T01:07:18.099Z,75,pass_forward,2495454,WR,away,2018090600,2495454,"(80.69, 44.91)",75,2555383,CB,home,None,None,"(76.53, 44.93)",4
10,2018-09-07T01:07:19.200Z,75,pass_arrived,2495454,WR,away,2018090600,2495454,"(81.11, 47.87)",75,2552689,CB,home,None,None,"(77.11, 17.3)",31
11,2018-09-07T01:07:19.200Z,75,pass_arrived,2495454,WR,away,2018090600,2495454,"(81.11, 47.87)",75,2555383,CB,home,None,None,"(74.4, 47.11)",7
30,2018-09-07T01:09:12.700Z,146,pass_forward,2560854,WR,away,2018090600,2560854,"(45.51, 36.91)",146,2552689,CB,home,None,None,"(38.1, 10.36)",28


In [529]:
#cuts df by half. Keeps only rows where it is WR then CB, and not CB and WR (which is just a repeat)
merged2_fixed = merged2_fixed[(merged2_fixed['position_x'] == 'WR')]
merged2_fixed.shape


(226, 17)

In [579]:
#self merging AGAIN, so now have a row with the pass_forward and the pass_arrived. Much easier to calculate diff
ForArrStack = merged2_fixed.merge(merged2_fixed, on = ['playId_x', 'nflId_x', 'nflId_y'], suffixes=['_Xagain', '_Yagain'])
ForArrStack.columns


Index(['time_Xagain', 'playId_x', 'event_Xagain', 'nflId_x',
       'position_x_Xagain', 'team_x_Xagain', 'gameId_x_Xagain',
       'targetNflId_x_Xagain', 'x,y_x_Xagain', 'playId_y_Xagain', 'nflId_y',
       'position_y_Xagain', 'team_y_Xagain', 'gameId_y_Xagain',
       'targetNflId_y_Xagain', 'x,y_y_Xagain', 'Euc_dist_Xagain',
       'time_Yagain', 'event_Yagain', 'position_x_Yagain', 'team_x_Yagain',
       'gameId_x_Yagain', 'targetNflId_x_Yagain', 'x,y_x_Yagain',
       'playId_y_Yagain', 'position_y_Yagain', 'team_y_Yagain',
       'gameId_y_Yagain', 'targetNflId_y_Yagain', 'x,y_y_Yagain',
       'Euc_dist_Yagain'],
      dtype='object')

In [580]:
#Get rid of rows where the forward time is equal to or greater than the arrived time
ForArrStack = ForArrStack[(ForArrStack['time_Xagain'] < ForArrStack['time_Yagain'])]

In [581]:
#cut down on useless rows
print(ForArrStack.iloc[10])
ForArrStack.drop(['time_Yagain', 'targetNflId_y_Yagain', 'position_x_Yagain', 'position_y_Yagain', 'gameId_y_Yagain', 'gameId_x_Yagain', 'playId_y_Yagain', 'gameId_y_Xagain', 'targetNflId_x_Yagain'], axis=1, inplace=True)
print(ForArrStack.shape)

time_Xagain             2018-09-07T01:16:54.900Z
playId_x                                     320
event_Xagain                        pass_forward
nflId_x                                  2552600
position_x_Xagain                             WR
team_x_Xagain                               home
gameId_x_Xagain                       2018090600
targetNflId_x_Xagain                     2552600
x,y_x_Xagain                      (16.54, 41.86)
playId_y_Xagain                              320
nflId_y                                  2556445
position_y_Xagain                             CB
team_y_Xagain                               away
gameId_y_Xagain                             None
targetNflId_y_Xagain                        None
x,y_y_Xagain                       (19.9, 19.39)
Euc_dist_Xagain                               23
time_Yagain             2018-09-07T01:16:55.700Z
event_Yagain                        pass_arrived
position_x_Yagain                             WR
team_x_Yagain       

In [555]:
#Calc the Euclidean Difference from pass forward and pass arrived
ForArrStack['Euc_diff'] = ForArrStack['Euc_dist_Xagain'] - ForArrStack['Euc_dist_Yagain']
ForArrStack.head()

,time_Xagain,playId_x,event_Xagain,nflId_x,position_x_Xagain,team_x_Xagain,gameId_x_Xagain,targetNflId_x_Xagain,"x,y_x_Xagain",playId_y_Xagain,...,targetNflId_y_Xagain,"x,y_y_Xagain",Euc_dist_Xagain,event_Yagain,team_x_Yagain,"x,y_x_Yagain",team_y_Yagain,"x,y_y_Yagain",Euc_dist_Yagain,Euc_diff
1,2018-09-07T01:07:18.099Z,75,pass_forward,2495454,WR,away,2018090600,2495454,"(80.69, 44.91)",75,...,None,"(79.51, 20.0)",25,pass_arrived,away,"(81.11, 47.87)",home,"(77.11, 17.3)",31,-6
5,2018-09-07T01:07:18.099Z,75,pass_forward,2495454,WR,away,2018090600,2495454,"(80.69, 44.91)",75,...,None,"(76.53, 44.93)",4,pass_arrived,away,"(81.11, 47.87)",home,"(74.4, 47.11)",7,-3
9,2018-09-07T01:09:12.700Z,146,pass_forward,2560854,WR,away,2018090600,2560854,"(45.51, 36.91)",146,...,None,"(38.1, 10.36)",28,pass_arrived,away,"(44.79, 36.79)",home,"(37.4, 9.64)",28,-1
13,2018-09-07T01:09:12.700Z,146,pass_forward,2560854,WR,away,2018090600,2560854,"(45.51, 36.91)",146,...,None,"(41.28, 38.93)",5,pass_arrived,away,"(44.79, 36.79)",home,"(43.3, 37.96)",2,3
17,2018-09-07T01:09:12.700Z,146,pass_forward,2560854,WR,away,2018090600,2560854,"(45.51, 36.91)",146,...,None,"(46.14, 25.29)",12,pass_arrived,away,"(44.79, 36.79)",home,"(45.65, 25.68)",11,0


In [569]:
#created a new DF and appended the CB nfl IDs and the Euclidean difference for each play
playerEucDiff = pd.DataFrame()
playerEucDiff['nflId_CB'] = ForArrStack['nflId_y']
playerEucDiff['Euc_diff'] = ForArrStack['Euc_diff']
playerEucDiff

,nflId_CB,Euc_diff
1,2552689,-6
5,2555383,-3
9,2552689,-1
13,2555383,3
17,2557958,0
21,2552689,9
25,2555383,-10
29,2557958,-2
33,2539334,-5
37,2539653,3


In [ ]:
#NOW. Need to go back and find which players, CBs, are closest to the targeted WR at the time of snap/pass forward,
#so only looking at that CB